# Test Small Patch Removal from Top Region

This notebook tests the preprocessing with small patch removal to verify it removes artifacts from the top region while keeping the large pin deck area.

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from src.lane_detection.preprocess_frames import preprocess_frame_hsv, remove_small_colored_patches_top, fill_small_black_patches

# Load a sample frame
video_path = "assets/input/cropped_test7.mp4"
cap = cv2.VideoCapture(video_path)

# Skip to frame 50 (where you saw issues)
cap.set(cv2.CAP_PROP_POS_FRAMES, 100)
ret, frame = cap.read()
cap.release()

print(f"Frame shape: {frame.shape}")
print(f"Frame loaded successfully: {ret}")

In [ ]:
# Step 1: HSV filtering
hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

# Brown mask
mask_brown = cv2.inRange(hsv_frame, (0, 0, 50), (20, 255, 255))

# Red/Orange mask  
mask_red_orange = cv2.inRange(hsv_frame, (150, 30, 200), (180, 200, 255))

# Combine masks
combined_mask = cv2.bitwise_or(mask_brown, mask_red_orange)

# Apply mask
masked_img = cv2.bitwise_and(frame, frame, mask=combined_mask)

print("HSV filtering complete")

In [ ]:
# Step 2: Fill small black patches
filled_img = fill_small_black_patches(masked_img, frame, max_patch_size_row=100, max_patch_size_col=50)

print("Gap filling complete")

In [ ]:
# Step 3: Test different thresholds for patch removal
thresholds = [500, 1000, 2000, 5000, 10000]

fig, axes = plt.subplots(2, 3, figsize=(20, 14))

# Original
axes[0, 0].imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
axes[0, 0].set_title('Original Frame', fontsize=14, fontweight='bold')
axes[0, 0].axis('off')

# After gap filling
axes[0, 1].imshow(cv2.cvtColor(filled_img, cv2.COLOR_BGR2RGB))
axes[0, 1].set_title('After Gap Filling', fontsize=14, fontweight='bold')
axes[0, 1].axis('off')

# Test different thresholds
for idx, threshold in enumerate(thresholds):
    result = remove_small_colored_patches_top(filled_img, top_region_ratio=0.3, max_patch_area=threshold)
    
    row = (idx + 2) // 3
    col = (idx + 2) % 3
    
    axes[row, col].imshow(cv2.cvtColor(result, cv2.COLOR_BGR2RGB))
    axes[row, col].set_title(f'MAX_AREA = {threshold}px\\n(Remove patches < {threshold})', 
                             fontsize=12, fontweight='bold')
    axes[row, col].axis('off')
    
    # Count patches in top region
    gray = cv2.cvtColor(result, cv2.COLOR_BGR2GRAY)
    height = gray.shape[0]
    top_height = int(height * 0.3)
    top_region = gray[:top_height, :]
    colored_mask = (top_region > 0).astype(np.uint8) * 255
    num_labels, _, stats, _ = cv2.connectedComponentsWithStats(colored_mask, connectivity=8)
    
    remaining_patches = num_labels - 1
    total_colored_pixels = np.sum(top_region > 0)
    
    print(f"Threshold {threshold:5d}px: {remaining_patches:2d} patches remaining, "
          f"{total_colored_pixels:6d} colored pixels in top region")

plt.tight_layout()
plt.savefig('output/patch_removal_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

print("\\n✓ Comparison saved to output/patch_removal_comparison.png")

In [ ]:
# Test with current config threshold (5000)
final_result = preprocess_frame_hsv(frame, max_patch_size_row=100, max_patch_size_col=50, 
                                    top_region_ratio=0.3, max_top_patch_area=5000)

# Visualize top region only
height = final_result.shape[0]
top_height = int(height * 0.3)

fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# Original top region
axes[0].imshow(cv2.cvtColor(frame[:top_height, :], cv2.COLOR_BGR2RGB))
axes[0].set_title('Original (Top 30%)', fontsize=14, fontweight='bold')
axes[0].axis('off')

# Before patch removal
axes[1].imshow(cv2.cvtColor(filled_img[:top_height, :], cv2.COLOR_BGR2RGB))
axes[1].set_title('After Gap Filling (Top 30%)', fontsize=14, fontweight='bold')
axes[1].axis('off')

# After patch removal
axes[2].imshow(cv2.cvtColor(final_result[:top_height, :], cv2.COLOR_BGR2RGB))
axes[2].set_title('After Patch Removal\\nThreshold=5000px (Top 30%)', fontsize=14, fontweight='bold')
axes[2].axis('off')

plt.tight_layout()
plt.savefig('output/top_region_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ Top region comparison saved to output/top_region_comparison.png")

In [ ]:
# Analyze patch sizes in top region BEFORE removal
gray_before = cv2.cvtColor(filled_img, cv2.COLOR_BGR2GRAY)
top_region_before = gray_before[:top_height, :]
colored_mask_before = (top_region_before > 0).astype(np.uint8) * 255
num_labels_before, labels_before, stats_before, _ = cv2.connectedComponentsWithStats(colored_mask_before, connectivity=8)

print("\n=== Patch Analysis BEFORE Removal ===")
print(f"Total patches in top region: {num_labels_before - 1}")
print("\nPatch sizes:")

areas_before = []
for label in range(1, num_labels_before):
    area = stats_before[label, cv2.CC_STAT_AREA]
    areas_before.append(area)
    if area < 100:
        print(f"  Patch {label}: {area} pixels (very small - will be removed)")
    elif area < 2000:
        print(f"  Patch {label}: {area} pixels (small - will be removed)")
    else:
        print(f"  Patch {label}: {area} pixels (large - will be KEPT)")

if areas_before:
    print(f"\nSmallest patch: {min(areas_before)} pixels")
    print(f"Largest patch: {max(areas_before)} pixels")
    print(f"Average patch: {np.mean(areas_before):.1f} pixels")
    print(f"Median patch: {np.median(areas_before):.1f} pixels")

# After removal
gray_after = cv2.cvtColor(final_result, cv2.COLOR_BGR2GRAY)
top_region_after = gray_after[:top_height, :]
colored_mask_after = (top_region_after > 0).astype(np.uint8) * 255
num_labels_after, labels_after, stats_after, _ = cv2.connectedComponentsWithStats(colored_mask_after, connectivity=8)

print("\n=== Patch Analysis AFTER Removal (threshold=2000px) ===")
print(f"Total patches remaining: {num_labels_after - 1}")
print(f"Patches removed: {(num_labels_before - 1) - (num_labels_after - 1)}")

if num_labels_after > 1:
    print("\nRemaining patch sizes:")
    for label in range(1, num_labels_after):
        area = stats_after[label, cv2.CC_STAT_AREA]
        print(f"  Patch {label}: {area} pixels (large pin deck area)")